In [2]:
!pip install faiss-cpu numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 117.7 MB/s eta 0:00:0000:0100:01


In [3]:
import pickle
import faiss
import numpy
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cloud_model_path = "/content/drive/MyDrive/models/"

In [6]:
model_path = "/content/drive/MyDrive/models/"
vectorizer = pickle.load(open(os.path.join(model_path, "transformed.pkl"), "rb"))
df = pickle.load(open(os.path.join(model_path, "df.pkl"), "rb"))

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
tfidf_matrix = vectorizer.transform(df["combined_columns"])
print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 94586 stored elements and shape (6976, 3232584)>
  Coords	Values
  (0, 53151)	0.0940480190212025
  (0, 53180)	0.19823566524066238
  (0, 77723)	0.14078673513507065
  (0, 77900)	0.2882474095183407
  (0, 412716)	0.1757142057424947
  (0, 450053)	0.3073970528868713
  (0, 450054)	0.3073970528868713
  (0, 689664)	0.1236156001610684
  (0, 690137)	0.3073970528868713
  (0, 1013510)	0.13840224026375328
  (0, 1013959)	0.2602998145006791
  (0, 1043550)	0.13441022401034805
  (0, 1494799)	0.12410140335198855
  (0, 1494897)	0.2190270051228177
  (0, 1509386)	0.18793382694024505
  (0, 1858330)	0.16200420833229726
  (0, 2186467)	0.2961952296140974
  (0, 2468466)	0.13467510710871286
  (0, 2468469)	0.13496944214525372
  (0, 2747744)	0.2517311346696327
  (0, 3131863)	0.19123360824790658
  (0, 3131887)	0.25568459534625415
  (1, 53151)	0.11073789809019946
  (1, 53180)	0.23341481430155792
  (1, 77723)	0.16577092521560288
  :	:
  (6974, 814716)	0.158

In [27]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=128, random_state=42)
embeddings = svd.fit_transform(tfidf_matrix).astype("float32")

In [28]:
faiss.normalize_L2(embeddings)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # inner product = cosine
index.add(embeddings)
faiss.write_index(index, "faiss_movies.index")
